In [15]:
# Import Subroutines:

include("../subroutines/Subroutines.jl");

3.10.9


In [2]:
using LinearAlgebra

using SparseArrays

I_4 =  sparse(Matrix(1.0I, 4, 4))

I_2 = sparse(Matrix(1.0I, 2, 2))

FSWAP_4 = sparse([
        1 0 0 0;
        0 0 1 0;
        0 1 0 0;
        0 0 0 -1
        ])

FSWAP_16 = kron(I_2,kron(FSWAP_4,I_2))*kron(FSWAP_4,FSWAP_4)*kron(I_2,kron(FSWAP_4,I_2))

dense_FSWAP = reshape(permutedims(reshape(Matrix(FSWAP_16), (4,4,4,4)), (1,3,2,4)), (16,16))

F = svd(dense_FSWAP)

lU = reshape(F.U, 4,4,16)
rV = reshape(F.V, 16,4,4)

qnvec = [
    QN(("Nf",0,-1),("Sz",0)) => 1,
    QN(("Nf",1,-1),("Sz",1)) => 1,
    QN(("Nf",1,-1),("Sz",-1)) => 1,
    QN(("Nf",2,-1),("Sz",0)) => 1,
    #---------------------------#
    QN(("Nf",-1,-1),("Sz",-1)) => 1,
    QN(("Nf",0,-1),("Sz",0)) => 1,
    QN(("Nf",0,-1),("Sz",-2)) => 1,
    QN(("Nf",1,-1),("Sz",-1)) => 1,
    #---------------------------#
    QN(("Nf",-1,-1),("Sz",1)) => 1,
    QN(("Nf",0,-1),("Sz",2)) => 1,
    QN(("Nf",0,-1),("Sz",0)) => 1,
    QN(("Nf",1,-1),("Sz",1)) => 1,
    #---------------------------#
    QN(("Nf",-2,-1),("Sz",0)) => 1,
    QN(("Nf",-1,-1),("Sz",1)) => 1,
    QN(("Nf",-1,-1),("Sz",-1)) => 1,
    QN(("Nf",0,-1),("Sz",0)) => 1
]

function PSWAP!(mpo, p)
    
    psite = [siteinds(mpo, plev=0)[p][1], siteinds(mpo, plev=0)[p+1][1]]
    
    plink = commoninds(mpo[p], mpo[p+1])
    
    nlink = Index(qnvec, tags="nlink")
    
    combo = combiner(plink, nlink, tags="link,l=$(p)")
    
    lswap = ITensor(lU, dag(setprime(psite[1],2)), setprime(psite[1],1), nlink)
    
    rswap = ITensor(rV, dag(nlink), dag(setprime(psite[2],2)), setprime(psite[2],1))
    
    mpo[p] = mpo[p] * lswap * combo
    
    mpo[p+1] = mpo[p+1] * rswap * dag(combo)
    
    setprime!(mpo[p], 1, plev=2)
    setprime!(mpo[p+1], 1, plev=2)
    
    #if maxlinkdim(mpo) > 1024
    truncate!(mpo, tol=1.0e-12)
    #end
    
end

PSWAP! (generic function with 2 methods)

In [3]:
N = 7

sites = siteinds("Electron", N, conserve_qns=true)

for pm=1:15
    
    ord1 = randperm(N)
    ord2 = randperm(N)
    ord3 = reverse(ord2)

    swap_inds1 = BubbleSort(ord1, ord2)
    swap_inds2 = BubbleSort(ord1, ord3)
    
    swap_inds = swap_inds1
    
    if length(swap_inds1) > length(swap_inds2)
        swap_inds = swap_inds2
    end

    pmpo = MPO(sites, "I")

    for p in swap_inds
        PSWAP!(pmpo, p)
    end

    println("$(pm):  $(maxlinkdim(pmpo))")#  $(maxlinkdim(pmpo2))")
end


1:  16
2:  256
3:  256
4:  256
5:  256
6:  256
7:  256
8:  256
9:  256
10:  256
11:  256
12:  16
13:  256
14:  256
15:  256


In [4]:
N_el = N

hf_occ = [FillHF(i, N_el) for i=1:N]

display(sites)

mps = randomMPS(sites, hf_occ, linkdims=1)

display(inner(pmpo, mps, pmpo, mps))

7-element Vector{Index{Vector{Pair{QN, Int64}}}}:
 (dim=4|id=91|"Electron,Site,n=1") <Out>
 1: QN(("Nf",0,-1),("Sz",0)) => 1
 2: QN(("Nf",1,-1),("Sz",1)) => 1
 3: QN(("Nf",1,-1),("Sz",-1)) => 1
 4: QN(("Nf",2,-1),("Sz",0)) => 1
 (dim=4|id=876|"Electron,Site,n=2") <Out>
 1: QN(("Nf",0,-1),("Sz",0)) => 1
 2: QN(("Nf",1,-1),("Sz",1)) => 1
 3: QN(("Nf",1,-1),("Sz",-1)) => 1
 4: QN(("Nf",2,-1),("Sz",0)) => 1
 (dim=4|id=155|"Electron,Site,n=3") <Out>
 1: QN(("Nf",0,-1),("Sz",0)) => 1
 2: QN(("Nf",1,-1),("Sz",1)) => 1
 3: QN(("Nf",1,-1),("Sz",-1)) => 1
 4: QN(("Nf",2,-1),("Sz",0)) => 1
 (dim=4|id=159|"Electron,Site,n=4") <Out>
 1: QN(("Nf",0,-1),("Sz",0)) => 1
 2: QN(("Nf",1,-1),("Sz",1)) => 1
 3: QN(("Nf",1,-1),("Sz",-1)) => 1
 4: QN(("Nf",2,-1),("Sz",0)) => 1
 (dim=4|id=71|"Electron,Site,n=5") <Out>
 1: QN(("Nf",0,-1),("Sz",0)) => 1
 2: QN(("Nf",1,-1),("Sz",1)) => 1
 3: QN(("Nf",1,-1),("Sz",-1)) => 1
 4: QN(("Nf",2,-1),("Sz",0)) => 1
 (dim=4|id=623|"Electron,Site,n=6") <Out>
 1: QN(("Nf",0,

LoadError: UndefVarError: pmpo not defined

In [5]:
using ITensors

qnvec = [QN(("Nf",0,-1),("Sz",0)) => 2,
    QN(("Nf",1,-1),("Sz",1)) => 4,
    QN(("Nf",1,-1),("Sz",-1)) => 4,
    QN(("Nf",2,-1),("Sz",0)) => 2]

ind = Index(qnvec)

ids = (dag(ind)', ind)

T = randomITensor(ids)

display(T[1,3])

#Q, R = qr(dense(T), (removeqns(ind), removeqns(ind)))

#display(Q.tensor)

0.0

In [6]:
Q,_,_ = svd(T, ind)

display(inds(Q))

((dim=12|id=210) <Out>
 1: QN(("Nf",0,-1),("Sz",0)) => 2
 2: QN(("Nf",1,-1),("Sz",1)) => 4
 3: QN(("Nf",1,-1),("Sz",-1)) => 4
 4: QN(("Nf",2,-1),("Sz",0)) => 2, (dim=12|id=55|"Link,u") <In>
 1: QN(("Nf",0,-1),("Sz",0)) => 2
 2: QN(("Nf",1,-1),("Sz",-1)) => 4
 3: QN(("Nf",1,-1),("Sz",1)) => 4
 4: QN(("Nf",2,-1),("Sz",0)) => 2)

In [11]:
display(flux(T))

T2 = onehot(ids[1]=>1, ids[2]=>2)

display(flux(T2))

QN(("Nf",0,-1),("Sz",0))

QN(("Nf",0,-1),("Sz",0))

In [16]:
using LinearAlgebra

mat1 = [
    0 2 4 3 1;
    3 0 4 1 2;
    4 6 0 3 2;
    1 8 4 0 3;
    2 1 4 2 0
]

mat1 += transpose(mat1)

display(mat1)

5×5 Matrix{Int64}:
 0   5   8  4  3
 5   0  10  9  3
 8  10   0  7  6
 4   9   7  0  5
 3   3   6  5  0

In [17]:
ChainMaxEls(mat1)

5×5 Matrix{Int64}:
  0   5   8   4   3
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
  4   9   7   0   5
  3   3   6   5   0

89
14


5×5 Matrix{Int64}:
  0   5   8   4   3
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
  3   3   6   5   0

85
15


5×5 Matrix{Int64}:
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
  3   3   6   5   0

35
55


5×5 Matrix{Int64}:
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1

32

In [2]:
sites = siteinds("Electron", 7)

psi = randomMPS(sites, [4,4,4,4,4,1,1], linkdims=6);

MPS
[1] ((dim=4|id=635|"Link,l=1"), (dim=4|id=434|"Electron,Site,n=1"))
[2] ((dim=4|id=387|"Electron,Site,n=2"), (dim=6|id=294|"Link,l=2"), (dim=4|id=635|"Link,l=1"))
[3] ((dim=4|id=560|"Electron,Site,n=3"), (dim=6|id=91|"Link,l=3"), (dim=6|id=294|"Link,l=2"))
[4] ((dim=4|id=107|"Electron,Site,n=4"), (dim=6|id=834|"Link,l=4"), (dim=6|id=91|"Link,l=3"))
[5] ((dim=4|id=268|"Electron,Site,n=5"), (dim=6|id=601|"Link,l=5"), (dim=6|id=834|"Link,l=4"))
[6] ((dim=4|id=86|"Electron,Site,n=6"), (dim=4|id=551|"Link,l=6"), (dim=6|id=601|"Link,l=5"))
[7] ((dim=4|id=840|"Electron,Site,n=7"), (dim=4|id=551|"Link,l=6"))


In [7]:
inner(psi, psi)

psi[2].tensor

Dim 1: (dim=4|id=387|"Electron,Site,n=2")
Dim 2: (dim=6|id=294|"Link,l=2")
Dim 3: (dim=4|id=635|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 4×6×4
[:, :, 1] =
  0.5291874801546498   -0.22382193213582868    …   0.00752221383387474
 -0.14101888395510803  -0.5706616710530368         0.0907757222993717
  0.03205053816743192  -0.15086007015194908       -0.12176969348653219
 -0.05437599528690669  -0.0020005808923655166      0.12591644936886665

[:, :, 2] =
  0.001627406747482893   0.12075401412459551   …   0.11919224220618471
 -0.14493195941743914   -0.004391303794599705     -0.059160823656155076
  0.2103128597422932    -0.07942546760916798      -0.269205909612074
 -0.17297474138267027   -0.3230861618063352        0.12229003994521956

[:, :, 3] =
  0.15329237939883314  -0.3476215785594093   …  -0.04986712598915079
  0.6426714886889915    0.11523676857696302      0.043239531161317374
  0.25558451396279275  -0.131903488673563        0.09658352135924997
 -0.1543398470609497    0.23442

In [38]:
function softmax(alpha, x)
    
    xmax = maximum(x)
    
    if minimum(x) <= 0
        println("Error: xmin <= 0")
        return nothing
    end
    
    exptot = 0.0
    
    for j=1:length(x)
        exptot += exp(alpha*x[j]/xmax)
    end
    
    return xmax/alpha*log(exptot)
    
end

softmax (generic function with 1 method)

In [82]:
x = [200]
println(x)

softmax(3.0, x)

[200]


200.0

In [1]:
using ITensors

In [2]:
i = Index(3)
j = Index(4)
k = Index(2)

(dim=2|id=381)

In [5]:
OH = onehot()

LoadError: MethodError: no method matching onehot(::Pair{Tuple{Index{Int64}, Index{Int64}, Index{Int64}}, Tuple{Int64, Int64, Int64}})
[0mClosest candidates are:
[0m  onehot([91m::Type{<:Number}[39m, [91m::Pair{<:Index}...[39m) at ~/.julia/packages/ITensors/LXBUp/src/itensor.jl:567
[0m  onehot([91m::Type{<:Number}[39m, [91m::Vector{<:Pair{<:Index}}[39m) at ~/.julia/packages/ITensors/LXBUp/src/itensor.jl:572
[0m  onehot([91m::Vector{<:Pair{<:Index}}[39m) at ~/.julia/packages/ITensors/LXBUp/src/itensor.jl:576
[0m  ...

In [7]:
A = randn((6,6))

H0 = A + transpose(A)

6×6 Matrix{Float64}:
  0.600237   0.643613  -2.3172     0.999158  -0.645821  -1.29218
  0.643613   4.34081   -1.28266    0.825289   1.30472   -0.344765
 -2.3172    -1.28266    0.158077  -0.460724   1.85856   -1.13678
  0.999158   0.825289  -0.460724  -1.24188   -2.57736   -0.951398
 -0.645821   1.30472    1.85856   -2.57736   -1.8938    -1.07846
 -1.29218   -0.344765  -1.13678   -0.951398  -1.07846   -0.909567

In [10]:
M = 6

A = randn((M,M))

H0 = A + transpose(A)

display(H0)

cols = [2,3,4]

for (i,c) in enumerate(cols)
    
    col = c-i+1
    
    H1 = zeros((M-i,M-i))
    H1[1:col-1,1:col-1] = H0[1:col-1,1:col-1]
    H1[1:col-1,col:end] = H0[1:col-1,col+1:end]
    H1[col:end,1:col-1] = H0[col+1:end,1:col-1]
    H1[col:end,col:end] = H0[col+1:end,col+1:end]
    
    H0 = H1
    
end
    
display(H0)


6×6 Matrix{Float64}:
  0.522764   0.267318   0.957764   -0.652565  -1.69027    2.76237
  0.267318  -2.2654    -1.29042     0.207895   2.18287   -1.13492
  0.957764  -1.29042    0.466027    0.299111  -1.05832    0.0421351
 -0.652565   0.207895   0.299111    1.58885   -0.838656  -0.630716
 -1.69027    2.18287   -1.05832    -0.838656  -2.72999   -2.14835
  2.76237   -1.13492    0.0421351  -0.630716  -2.14835   -0.942323

3×3 Matrix{Float64}:
  0.522764  -1.69027   2.76237
 -1.69027   -2.72999  -2.14835
  2.76237   -2.14835  -0.942323

# 